#   Basic Model 1

In [1]:
import pandas as pd 
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
import textwrap
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\reliance\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reliance\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df = pd.read_csv("bbc-text.csv")

In [6]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [7]:
def summerize(doc):
    tokenized_doc = sent_tokenize(doc)
    sentences = tokenized_doc
    df = pd.DataFrame(sentences,columns=['sentence'])
    vectorizer = TfidfVectorizer(max_features=200)
    num_col = vectorizer.fit_transform(df['sentence'])
#     return num_col.todense()
    idx_score = {}
    for idx , score in enumerate(num_col.todense()):
        avg_score = np.average(score)
        if (idx and avg_score) not in idx_score.items():
            idx_score[idx] = avg_score
    sorted_scores = sorted(idx_score.values(),reverse=True)
    
    def find_key(d , target_value):
        for k , v in d.items():
            if v == target_value:
                return k
    print("Most Common sentences from the Document: ")
    print()
    i=0
    while(i<5):
        score = sorted_scores[i]
        key = find_key(idx_score,score)
        print(df.iloc[key]['sentence'],end=" ")
        
        i+=1

In [8]:
summerize(df['text'][23])

Most Common sentences from the Document: 

the return to fitness of darius vassell after four months out with a broken ankle and the emergence of luke moore has alleviated some of the villa s manager s problems in that department. the striker  who is on a season-long loan from chelsea  picked up the knock in an england under-21 match against holland earlier this month. cole  who also struggled with an ankle problem earlier in the season  was unable to rest because o leary had a shortage of strikers. carlton will be out of action for four to six weeks after a bad challenge   said villa boss david o leary. i won t be able to tell you whether he will need an operation until maybe next week. 

In [9]:
df['text'][23]

'cole faces lengthy injury lay-off aston villa s carlton cole could be out for six weeks with a knee injury.  the striker  who is on a season-long loan from chelsea  picked up the knock in an england under-21 match against holland earlier this month.  carlton will be out of action for four to six weeks after a bad challenge   said villa boss david o leary.  i won t be able to tell you whether he will need an operation until maybe next week. whether he has an operation has got to be left to chelsea.  cole  who also struggled with an ankle problem earlier in the season  was unable to rest because o leary had a shortage of strikers. the return to fitness of darius vassell after four months out with a broken ankle and the emergence of luke moore has alleviated some of the villa s manager s problems in that department.'

# More Sophisticated TF-IDF Approach

In [10]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [11]:
# One document from the entire dataset
doc = df[df.category == "entertainment"]['text'].sample(random_state=101)

In [12]:
doc

230    tv show unites angolan families angolan famili...
Name: text, dtype: object

In [13]:
# For utilizing the space very efficiently
def wrap(x):
    return textwrap.fill(x , replace_whitespace = False , fix_sentence_endings = True)

In [14]:
print(wrap(doc.iloc[0]))

tv show unites angolan families angolan families who are attempting to
track each other down  after being separated by nearly 30 years of war
are succeeding thanks to a hugely popular tv show.  meeting point has
become one of tv angola s most watched programmes  and has reunited
hundreds of families.  it runs daily  not only on the television but
also on the radio.  every friday  hundreds of people gather in luanda
s independence square to record a message in front of the tv cameras
in the hope that a lost relative will see it.  many relatives have
been reunited on air.  at the beginning there was an absolute
explosion - huge  huge crowds   sergio gera  the programme s chief co-
ordinator  told bbc world service s assignment programme.  now things
are a little calmer  there are slightly less people - but  after two
and a half years of broadcasting  there are still a lot of people
going.   the media in the southern african country  twice the size of
france  has been gripped by the quest

In [15]:
# getting the sentences from the document
sents = sent_tokenize(doc.iloc[0])

In [16]:
# Vectorization
tf_idf = TfidfVectorizer(
                stop_words="english",
                norm="l1")

In [17]:
X = tf_idf.fit_transform(sents)

In [18]:
# Put the avg_score of each sentence
avg_score = np.zeros(len(sents))
def score(vector):
    mat = vector.todense()
    new_mat = []
    for i in mat:
        new_mat.append(i[i!=0])
    for i in range(len(new_mat)):
        avg_score[i] = new_mat[i].mean()

In [19]:
score(X)

In [20]:
sorted_idx = np.argsort(avg_score)

In [23]:
for i in sorted_idx[0:3]:
    print(sents[i],end=" ")

the programme of family reunification is extremely important  but for these families to remain reunified  there has to be social integration  job access  education  healthcare   said rafael marques of the pro-democracy george soros foundation for southern africa. in the city of wambo  the icrc runs the gazetta - a 200-page  tabloid-size book which contains 13 000 names of missing or displaced. at the beginning there was an absolute explosion - huge  huge crowds   sergio gera  the programme s chief co-ordinator  told bbc world service s assignment programme. 